In [1]:
import geojson
from geojson import Polygon, FeatureCollection, Feature, MultiPolygon
import pandas as pd
import json
from random import randint

In [2]:
# import folium

# map_2 = folium.Map(location=[39.2904, -76.6122],
#                    zoom_start=11)
# for s in parking_citations_data_withgeo["GeoPoint"].values[0:60]:
#     try:
#         s1 = s[1:len(s)-1].split(",")
#         latitude = float(s1[0].strip())
#         longitude = float(s1[1].strip())
#         folium.CircleMarker([latitude, longitude],
#                     radius=1,
#                     color='#3186cc',
#                     fill_color='#3186cc',
#                    ).add_to(map_2)
#     except:
#         print(s1)
# map_2

In [3]:
baltimore_city_geojson_string = open("data/Baltimore City Line.geojson").read()
baltimore_city_geojson = geojson.loads(baltimore_city_geojson_string)

minLat = float("inf")
maxLat = float("-inf")
minLong = float("inf")
maxLong = float("-inf")

#Longitude, Latitude
for coordinate in baltimore_city_geojson["features"][0]["geometry"]["coordinates"][0]:
    lat = coordinate[1]
    if(lat < minLat):
        minLat = lat
    if(lat > maxLat):
        maxLat = lat
    long = coordinate[0]
    if(long < minLong):
        minLong = long
    if(long > maxLong):
        maxLong = long

baltimore_geo_extremes = {
    "minLatitude" : minLat,
    "maxLatitude" : maxLat,
    "minLongitude" : minLong,
    "maxLongitude" : maxLong
}

with open('modified_data/geo_extremes.json', 'w') as outfile:
    json.dump(baltimore_geo_extremes, outfile)

In [4]:
print(baltimore_city_geojson)

{"features": [{"geometry": {"coordinates": [[[-76.71129366997027, 39.37195703067297], [-76.52967423510157, 39.37197190004332], [-76.52985830094921, 39.20962295330451], [-76.54972531264977, 39.197233450625134], [-76.58367312662826, 39.20812053179621], [-76.61161075881019, 39.23439454752913], [-76.71116134911031, 39.27783849660702], [-76.71129366997027, 39.37195703067297]]], "type": "MultiLineString"}, "properties": {"area": "2562713344", "boundary_": "2.0", "boundary_i": "0.0", "id": "0.0", "length": "207126.513", "perimeter": "207126.51563", "shape_leng": "207126.513062"}, "type": "Feature"}], "type": "FeatureCollection"}


In [5]:
MultiPolygon([
    ([(3.78, 9.28), (-130.91, 1.52), (35.12, 72.234), (3.78, 9.28)],),
    ([(23.18, -34.29), (-1.31, -4.61), (3.41, 77.91), (23.18, -34.29)],)
])

{"coordinates": [[[[3.78, 9.28], [-130.91, 1.52], [35.12, 72.234], [3.78, 9.28]]], [[[23.18, -34.29], [-1.31, -4.61], [3.41, 77.91], [23.18, -34.29]]]], "type": "MultiPolygon"}

In [6]:
numBins = 25
longBinSize = abs(maxLong - minLong)/float(numBins)
latBinSize = abs(maxLat - minLat)/float(numBins)
polygonlist = []
boxNum = 0
for latI in range(0, numBins):
    smallLat = minLat + latI*latBinSize
    bigLat = minLat + (latI+1)*latBinSize
    for longI in range(0, numBins):
        smallLong = minLong + longI*longBinSize
        bigLong = minLong + (longI+1)*longBinSize
        color = "rgb({}, {}, {})".format(randint(0, 255), randint(0, 255), randint(0, 255))
        props = {"boxNumber": str(boxNum), "fill" : color, "stroke": color, "fill-opacity": 1.0, "stroke-width":"1"}
        polygonlist.append(Feature(id = boxNum, properties = props, geometry = Polygon([[(smallLong, smallLat),(smallLong, bigLat),(bigLong, bigLat),(bigLong, smallLat)]])))
        boxNum += 1
boxed_baltimore = FeatureCollection(polygonlist)
boxed_baltimore

{"features": [{"geometry": {"coordinates": [[[-76.71129366997027, 39.197233450625134], [-76.71129366997027, 39.20422298860186], [-76.70402889257552, 39.20422298860186], [-76.70402889257552, 39.197233450625134]]], "type": "Polygon"}, "id": 0, "properties": {"boxNumber": "0", "fill": "rgb(6, 127, 219)", "fill-opacity": 1.0, "stroke": "rgb(6, 127, 219)", "stroke-width": "1"}, "type": "Feature"}, {"geometry": {"coordinates": [[[-76.70402889257552, 39.197233450625134], [-76.70402889257552, 39.20422298860186], [-76.69676411518078, 39.20422298860186], [-76.69676411518078, 39.197233450625134]]], "type": "Polygon"}, "id": 1, "properties": {"boxNumber": "1", "fill": "rgb(80, 233, 109)", "fill-opacity": 1.0, "stroke": "rgb(80, 233, 109)", "stroke-width": "1"}, "type": "Feature"}, {"geometry": {"coordinates": [[[-76.69676411518078, 39.197233450625134], [-76.69676411518078, 39.20422298860186], [-76.68949933778603, 39.20422298860186], [-76.68949933778603, 39.197233450625134]]], "type": "Polygon"}, "

In [7]:
with open('modified_data/boxed_baltimore.geojson', 'w') as boxedBaltimoreFile:
    boxedBaltimoreGeoJsonString = geojson.dumps(boxed_baltimore)
    boxedBaltimoreFile.write(boxedBaltimoreGeoJsonString)
    boxedBaltimoreFile.close()